In [1]:
from time import time
from enum import Enum

import nest_asyncio
import pandas as pd
import numpy as np
import tensorflow as tf
from pathlib import Path

nest_asyncio.apply()
SEED = 1337
tf.random.set_seed(SEED)
import gc

gc.collect()


2024-06-20 14:39:56.129989: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 14:39:56.133967: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 14:39:56.176683: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-20 14:39:57.128760: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


0

In [2]:
this_dir = Path.cwd()
print(this_dir)

# NUM_FEATURE = len(data_test_without_flow.columns) - 1
NUM_FEATURE = 512
NUM_PACKETS_PER_FLOW = 15
EPOCHS = 5
NUM_CLASSES = 3

EXPERIMENT_NAME = f"RUN-{NUM_PACKETS_PER_FLOW}-{NUM_FEATURE}"


MODEL_PATH = this_dir / "results" / "model.keras"
# MODEL_PATH = this_dir / "results" / f"DED-{NUM_PACKETS_PER_FLOW}-{NUM_FEATURE}-NEW" / "model.keras"

print("Model path: ",  MODEL_PATH)

print('Number of features: ', NUM_FEATURE, ', Number of classes: ', NUM_CLASSES, ', Number of packets per flow: ',
      NUM_PACKETS_PER_FLOW)

/home/bkcs/Documents/doh/doh-exfiltration-detector
Model path:  /home/bkcs/Documents/doh/doh-exfiltration-detector/results/model.keras
Number of features:  512 , Number of classes:  3 , Number of packets per flow:  15


# Load data

In [3]:

data_dir = this_dir


class Label(Enum):
    benign = 0
    dnscat2 = 1 
    # dns2tcp = 2
    iodine = 2

label_str = 'iodine'

data = pd.read_csv(data_dir / 'dataset' / "iodine_real_1111.csv", nrows=None)
# data = pd.read_csv(data_dir / "tung" / f"{label_str}" / f"out.csv")
data['label'] = Label[label_str].value

print(str(data.groupby('flow_id').size().count()))

252


# Get label

In [4]:
def most_frequent(flow):
    return max(set(flow), key=flow.count)

In [5]:
def get_label(data):
    grouped = data.groupby('flow_id')['label'].apply(list).to_dict()

    label = []
    for flow in grouped:
        label.append(most_frequent(grouped[flow]))

    return np.array(label)

In [6]:
label_test = np.array(get_label(data))

print('Label: ', len(label_test))

Label:  252


In [7]:
# Drop number of rows per flow to NUM_PACKETS_PER_FLOW
def drop_rows(data):
    return data.groupby('flow_id').head(NUM_PACKETS_PER_FLOW)

data = drop_rows(data)

In [8]:
data_without_flow = data.drop('flow_id', axis=1)

In [9]:
x_test = np.expand_dims(
    (data_without_flow.drop('label', axis=1).iloc[:, :NUM_FEATURE].to_numpy() / 255).reshape(-1, NUM_PACKETS_PER_FLOW, NUM_FEATURE),
    axis=-1)

print('x_test shape: ', x_test.shape)

x_test shape:  (252, 15, 512, 1)


# Load the model

In [10]:
model = tf.keras.models.load_model(MODEL_PATH)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 15, 512, 32)    │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 15, 512, 32)    │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 7, 256, 32)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 7, 256, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 256, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 7, 256, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 3, 128, 64)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 3, 128, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 24576)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,291,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,123,115 (72.95 MB)

 Trainable params: 6,374,371 (24.32 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 12,748,744 (48.63 MB)

# Evaluate the model

In [11]:
def sec_to_hours(seconds):
    a = seconds // 3600
    b = (seconds % 3600) // 60
    c = (seconds % 3600) % 60
    d = "{:.0f} hours {:.0f} mins {:.0f} seconds".format(a, b, c)
    return d

In [12]:
client_lr = 3e-4
NUM_ROUNDS = 300
BATCH_SIZE = 4

start = time()
predictions = model.predict(
    x_test, verbose=2, batch_size=BATCH_SIZE)
end = time() - start

I0000 00:00:1718869199.478734   62516 service.cc:145] XLA service 0x7fe5d00081d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1718869199.478767   62516 service.cc:153]   StreamExecutor device (0): Host, Default Version
2024-06-20 14:39:59.486810: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1718869199.584647   62516 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


63/63 - 2s - 26ms/step


In [13]:
flow_pred = np.argmax(predictions, axis=-1)

In [14]:
label_test

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [15]:
flow_pred

array([2, 2, 2, 1, 1, 2, 1, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 0,
       1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 0, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1,
       2, 2, 1, 1, 2, 1, 2, 2, 0, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1,
       1, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 1, 2,
       2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2,
       2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 1, 2, 1, 1, 2, 2, 2,
       1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2, 1, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 0, 1, 1, 2,
       1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 2, 2, 2, 2])

In [16]:
print(str((label_test == flow_pred).sum()/float(flow_pred.size)*100) + '%')

68.65079365079364%


In [26]:
# GET unique value in np array, then calculate the percentage of each value
unique, counts = np.unique(flow_pred, return_counts=True)
print(np.asarray((unique, counts)).T)

for value in unique:
    print(f'Percentage of {value}: {counts[value]/len(flow_pred)*100}%')


[[  0   9]
 [  1  70]
 [  2 173]]
Percentage of 0: 3.571428571428571%
Percentage of 1: 27.77777777777778%
Percentage of 2: 68.65079365079364%
